### Install packages and perform maintance

In [2]:
import gc

from tqdm import tqdm
import numpy as np
import pandas as pd

import torch

In [3]:
# Cuda maintenance
gc.collect()
torch.cuda.empty_cache()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Torch device: ", device)

Torch device:  cuda


In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 34.5 MB/s 
     |████████████████████████████████| 163 kB 72.1 MB/s 
     |████████████████████████████████| 212 kB 71.8 MB/s 
     |████████████████████████████████| 115 kB 70.4 MB/s 
     |████████████████████████████████| 95 kB 5.0 MB/s 
     |████████████████████████████████| 127 kB 73.4 MB/s 
     |████████████████████████████████| 115 kB 72.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 19.6 MB/s 
     |████████████████████████████████| 7.6 MB 21.8 MB/s 


In [6]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 88 kB/s 


In [7]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

### Load and explore dataset

https://huggingface.co/datasets/sberquad


In [9]:
from datasets import load_dataset

# importing dataset
sberquad_dataset = load_dataset("sberquad")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

Dataset sberquad downloaded and prepared to /root/.cache/huggingface/datasets/sberquad/sberquad/1.0.0/62115d937acf2634cfacbfee10c13a7ee39df3ce345bb45af7088676f9811e77. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
print(sberquad_dataset["train"]["context"][0])
print("Question: " + sberquad_dataset["train"]["question"][0])
print(sberquad_dataset["train"]["answers"][0])

В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.
Question: чем представлены органические остатки?
{'text': ['известковыми выделениями сине-зелёных водорослей'], 'answer_start': [109]}


### Tokenize datasets


In [11]:
# https://huggingface.co/DeepPavlov/rubert-base-cased

checkpoint = "DeepPavlov/rubert-base-cased"

Firstly, was chosen  "AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru" model.
However, it required more space on GPU during training.




In [12]:
# Training preprocessing. Source: https://huggingface.co/course/chapter7/7?fw=pt
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def preprocess_training_examples(examples, max_seq_length=384, stride=128):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(questions,
                       examples["context"],
                       max_length=max_seq_length,
                       truncation="only_second",
                       return_offsets_mapping=True,
                       padding="max_length")

    offset_mapping = inputs["offset_mapping"]
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [13]:
tokenized_dataset = sberquad_dataset.map(preprocess_training_examples, batched=True)

  0%|          | 0/46 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

In [14]:
import torch
from sklearn.model_selection import train_test_split

validation_idx, test_idx = train_test_split(list(range(len(tokenized_dataset['validation']))),
                                     test_size=0.6, random_state=42)

validation_dataset = torch.utils.data.Subset(tokenized_dataset['validation'], validation_idx)
test_dataset = torch.utils.data.Subset(tokenized_dataset['validation'], test_idx)

### Load model and define function for metrics calculation


In [15]:
import evaluate

squad_metric = evaluate.load("squad")

def compute_metrics(eval_preds, dataset, metric):
    y_pred = np.argmax(eval_preds[0], -1).T

    f1_score = 0
    exact_match = 0
    for data, pred in zip(dataset, y_pred):
        # Convert answer start and end into characters positions in
        # original text using the offset mapping list
        start_char = data['offset_mapping'][pred[0]][0]
        end_char = data['offset_mapping'][pred[1]][1]
        # Create predictions and references dictionaries for metric function
        predictions = [{'prediction_text': data['context'][start_char:end_char],
                        'id': str(data['id'])}]
        references = [{'answers': data['answers'],
                        'id': str(data['id'])}]
        results = metric.compute(predictions=predictions,
                                  references=references)
        # Add metric to running sum variable to calculate average after,
        # change outputs from 0-100 range to 0-1 range
        f1_score += results['f1'] / 100
        exact_match += results['exact_match'] / 100
    # Calculate the average
    f1_score /= len(dataset)
    exact_match /= len(dataset)
    return {'f1': f1_score, 'exact_match': exact_match}

In [16]:
from transformers import  AutoModelForQuestionAnswering, Trainer
import torchinfo

model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)
torchinfo.summary(model, input_data=torch.randint(low=0, high=30522, size=(1, 384)))

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from 

Layer (type:depth-idx)                                  Output Shape              Param #
BertForQuestionAnswering                                [1, 384]                  --
├─BertModel: 1-1                                        [1, 384, 768]             --
│    └─BertEmbeddings: 2-1                              [1, 384, 768]             --
│    │    └─Embedding: 3-1                              [1, 384, 768]             91,812,096
│    │    └─Embedding: 3-2                              [1, 384, 768]             1,536
│    │    └─Embedding: 3-3                              [1, 384, 768]             393,216
│    │    └─LayerNorm: 3-4                              [1, 384, 768]             1,536
│    │    └─Dropout: 3-5                                [1, 384, 768]             --
│    └─BertEncoder: 2-2                                 [1, 384, 768]             --
│    │    └─ModuleList: 3-6                             --                        85,054,464
├─Linear: 1-2                    

### Define training args and train model

In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir='./',
                                  report_to='none',
                                  evaluation_strategy="epoch",
                                  optim="adamw_torch",
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  warmup_steps=200,
                                  lr_scheduler_type='cosine',
                                  num_train_epochs=3,
                                  save_total_limit = 2, # save only best and last
                                  save_strategy = "epoch",
                                  load_best_model_at_end=True,
                                  metric_for_best_model='eval_f1')

In [18]:
compute_squad_metrics = lambda eval_preds: compute_metrics(eval_preds, dataset=validation_dataset, metric=squad_metric)

In [19]:
trainer = Trainer(model,
                  training_args,
                  train_dataset=tokenized_dataset['train'],
                  eval_dataset=validation_dataset,
                  compute_metrics=compute_squad_metrics)

In [20]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: question, answers, context, id, title, offset_mapping. If question, answers, context, id, title, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 45328
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 8499
  Number of trainable parameters = 177264386


Epoch,Training Loss,Validation Loss,F1,Exact Match
1,1.687200,1.580957,0.785686,0.571996
2,1.338400,1.541804,0.770321,0.553128
3,1.083700,1.622365,0.743712,0.527805


***** Running Evaluation *****
  Num examples = 2014
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: question, answers, context, id, title, offset_mapping. If question, answers, context, id, title, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
Saving model checkpoint to ./checkpoint-2833
Configuration saved in ./checkpoint-2833/config.json
Model weights saved in ./checkpoint-2833/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2014
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: question, answers, context, id, title, offset_mapping. If question, answers, context, id, title, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
Saving mo

TrainOutput(global_step=8499, training_loss=1.502056918518447, metrics={'train_runtime': 10491.5187, 'train_samples_per_second': 12.961, 'train_steps_per_second': 0.81, 'total_flos': 2.664913362599117e+16, 'train_loss': 1.502056918518447, 'epoch': 3.0})

In [21]:
compute_squad_test_metrics = lambda eval_preds: compute_metrics(eval_preds, dataset=test_dataset, metric=squad_metric)

model.eval()
with torch.inference_mode():
    start_logits, end_logits = [], []
    for item in tqdm(test_dataset):
        input_ids = torch.tensor(item['input_ids']).unsqueeze(0).to('cuda')
        attention_mask = torch.tensor(item['attention_mask']).unsqueeze(0).to('cuda')
        token_type_ids = torch.tensor(item['token_type_ids']).unsqueeze(0).to('cuda')

        output = model(input_ids=input_ids,
                       token_type_ids=token_type_ids,
                       attention_mask=attention_mask)

        start_logits.append(output['start_logits'].squeeze(0).cpu().numpy())
        end_logits.append(output['end_logits'].squeeze(0).cpu().numpy())
 
    predictions = ((np.array(start_logits), np.array(end_logits)), -1)

metrics = compute_squad_test_metrics(predictions)
print(f'\nF1-score: {metrics["f1"]}')

100%|██████████| 3022/3022 [01:25<00:00, 35.54it/s]



F1-score: 0.7997223331719344


### Show predicted examples

In [22]:
predictions, _, _ = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 3022
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: question, answers, context, id, title, offset_mapping. If question, answers, context, id, title, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.


In [42]:
start_logits, end_logits = predictions

In [43]:
idx = 10
test_example = test_dataset[idx]
example_id = test_example["id"]
context = test_example["context"]
question = test_example["question"]
start_y_pred = np.argmax(start_logits[idx], -1).T
end_y_pred = np.argmax(end_logits[idx], -1).T
start_char = test_example['offset_mapping'][start_y_pred][0]
end_char = test_example['offset_mapping'][end_y_pred][1]

In [44]:
question

'Когда Нансен посетил Копенгаген?'

In [45]:
context

'В защиту Нансена встали в основном датские специалисты, например, известный полярник и знаток Гренландии Хенрик Йохан Ринк (1819—1893), создатель теории ледниковой эпохи, впервые предположивший, что гренландский ледяной щит есть последний реликт четвертичного оледенения. Ринк же преподавал Нансену и его спутникам основы гренландского языка[60]. В прессе в поддержку Нансена публично выступил геолог — профессор Амунд Теодор Хелланд (1846—1918), его статья обратила на себя внимание датского предпринимателя Августина Гаме́ля (1839—1904), который уже 12 января 1888 года предоставил Нансену искомые 5000 крон. Нансен с радостью принял дар, за что подвергся нападкам норвежских патриотов[61]. Во время посещения Копенгагена 2 мая 1888 года Нансен встретился с Кристианом Майгором — спутником Роберта Пири в его попытке пересечь Гренландию в 1886 году. Майгор заверил Нансена, что ничего невозможного в его предприятии нет[62].'

Answer:

In [46]:
context[start_char:end_char]

'2 мая 1888 года'